In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision as tv

import os
import numpy as np
import yaml
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

from torch.cuda.amp import autocast, GradScaler

In [18]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    print(device)

cuda


In [19]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import json

from PIL import Image


class EmoSet(Dataset):
    ATTRIBUTES_MULTI_CLASS = [
        'scene', 'facial_expression', 'human_action', 'brightness', 'colorfulness',
    ]
    ATTRIBUTES_MULTI_LABEL = [
        'object'
    ]
    NUM_CLASSES = {
        'brightness': 11,
        'colorfulness': 11,
        'scene': 254,
        'object': 409,
        'facial_expression': 6,
        'human_action': 264,
    }

    def __init__(self,
                 data_root,
                 num_emotion_classes,
                 phase,
                 ):
        assert num_emotion_classes in (8, 2)
        assert phase in ('train', 'val', 'test')
        self.transforms_dict = self.get_data_transforms()

        self.info = self.get_info(data_root, num_emotion_classes)
        if phase == 'train':
            self.transform = self.transforms_dict['train']
        elif phase == 'val':
            self.transform = self.transforms_dict['val']
        elif phase == 'test':
            self.transform = self.transforms_dict['test']
        else:
            raise NotImplementedError

        data_store = json.load(open(os.path.join(data_root, f'{phase}.json')))
        self.data_store = [
            [
                self.info['emotion']['label2idx'][item[0]],
                item[1],
                os.path.join(data_root, item[2]),
                os.path.join(data_root, item[3])
            ]
            for item in data_store
        ]

    @classmethod
    def get_data_transforms(cls):
        transforms_dict = {
            'train': transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                
            ]),

            'val': transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
        }
        return transforms_dict

    def get_info(self, data_root, num_emotion_classes):
        assert num_emotion_classes in (8, 2)
        info = json.load(open(os.path.join(data_root, 'info.json')))
        if num_emotion_classes == 8:
            pass
        elif num_emotion_classes == 2:
            emotion_info = {
                'label2idx': {
                    'amusement': 0,
                    'awe': 0,
                    'contentment': 0,
                    'excitement': 0,
                    'anger': 1,
                    'disgust': 1,
                    'fear': 1,
                    'sadness': 1,
                },
                'idx2label': {
                    '0': 'positive',
                    '1': 'negative',
                }
            }
            info['emotion'] = emotion_info
        else:
            raise NotImplementedError
        return info

    def load_image_by_path(self, path):
        image = Image.open(path).convert('RGB')
        image = self.transform(image)
        return image

    def load_annotation_by_path(self, path):
        json_data = json.load(open(path))
        return json_data

    def __getitem__(self, item):
        emotion_label_idx = torch.tensor(self.data_store[item][0])
        image_id, image_path, annotation_path = self.data_store[item][1], self.data_store[item][2], self.data_store[item][3]
        image = self.load_image_by_path(image_path)
        annotation_data = self.load_annotation_by_path(annotation_path)
        data = {'image_id': image_id, 'image': image, 'emotion_label_idx': emotion_label_idx}
        for attribute in self.ATTRIBUTES_MULTI_CLASS:
            attribute_label_idx = -1
            if attribute in annotation_data:
                attribute_label_idx = self.info[attribute]['label2idx'][str(annotation_data[attribute])]
            data.update({f'{attribute}_label_idx': attribute_label_idx})
        for attribute in self.ATTRIBUTES_MULTI_LABEL:
            assert attribute == 'object'
            num_classes = self.NUM_CLASSES[attribute]
            attribute_label_idx = torch.zeros(num_classes)
            if attribute in annotation_data:
                for label in annotation_data[attribute]:
                    attribute_label_idx[self.info[attribute]['label2idx'][label]] = 1
            data.update({f'{attribute}_label_idx': attribute_label_idx})
        return data

    def __len__(self):
        return len(self.data_store)


if __name__ == '__main__':
    data_root = r'C:\EmoSet\EmoSet-118K'
    num_emotion_classes = 8
    phase = 'train'
    
    dataset = EmoSet(
        data_root=data_root,
        num_emotion_classes=num_emotion_classes,
        phase=phase,
    )
    dataloader = DataLoader(dataset, batch_size = 16, shuffle = True)
    for i, data in enumerate(dataloader):
        print(i)
        print(data['image_id'])
        print(data['image'])
        print(data['emotion_label_idx'])
        print(data['scene_label_idx'])
        print(data['facial_expression_label_idx'])
        print(data['human_action_label_idx'])
        print(data['brightness_label_idx'])
        print(data['colorfulness_label_idx'])
        print(data['object_label_idx'])
        break

0
['excitement_12594', 'sadness_01828', 'excitement_10900', 'anger_06467', 'anger_04209', 'fear_02299', 'amusement_05149', 'anger_01396', 'excitement_08459', 'anger_10270', 'awe_09312', 'excitement_09058', 'amusement_05800', 'excitement_14975', 'awe_08880', 'excitement_08962']
tensor([[[[-0.9192, -0.9020, -0.9020,  ..., -1.3644, -1.3815, -1.3815],
          [-0.9020, -0.8849, -0.8849,  ..., -1.3644, -1.3473, -1.3473],
          [-0.9020, -0.8849, -0.8678,  ..., -1.3473, -1.3473, -1.3473],
          ...,
          [-0.2684, -0.2684, -0.2684,  ..., -1.1418, -1.1589, -1.1760],
          [-0.2856, -0.2684, -0.2684,  ..., -1.1589, -1.1760, -1.1760],
          [-0.2856, -0.2684, -0.2684,  ..., -1.1589, -1.1760, -1.1932]],

         [[-0.0924, -0.0749, -0.0749,  ..., -0.5651, -0.5826, -0.5826],
          [-0.0749, -0.0574, -0.0574,  ..., -0.5651, -0.5476, -0.5476],
          [-0.0749, -0.0574, -0.0399,  ..., -0.5476, -0.5476, -0.5476],
          ...,
          [ 0.5553,  0.5553,  0.5553,  ...

In [20]:
data_root = r'C:\EmoSet\EmoSet-118K'
num_emotion_classes = 8

phase = 'train'   
train_dataset = EmoSet(
    data_root=data_root,
    num_emotion_classes=num_emotion_classes,
    phase=phase,
)
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)

phase = 'val'  
val_dataset = EmoSet(
    data_root=data_root,
    num_emotion_classes=num_emotion_classes,
    phase=phase,
)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = True)

phase = 'test'
test_dataset = EmoSet(
    data_root=data_root,
    num_emotion_classes=num_emotion_classes,
    phase=phase,
)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = True)

In [21]:
for sample in train_loader:
    print(sample['image'].shape)
    break

torch.Size([16, 3, 224, 224])


In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [34]:
import copy
def test(val_loader,model,device,criterion):
    model.eval()
    epoch_val_accuracy,epoch_val_loss = 0,0 
    with torch.no_grad():
        for sample in val_loader:
            X, y = sample[0], sample[1]
            X = X.to(device)
            y = y.to(device)
            
            val_output = model(X)
            if criterion is None:
                val_loss = F.nll_loss(val_output, y, reduction='sum').item()
            else:
                val_loss = criterion(val_output, y)
            
            acc = (val_output.argmax(dim=1) == y).float().mean()
            epoch_val_accuracy += acc 
            epoch_val_loss += val_loss
    return epoch_val_accuracy/ len(val_loader),epoch_val_loss/ len(val_loader)

def train(train_loader,val_loader,model,device,n_epochs, optimizer,criterion,scheduler):
    train_accuracies,val_accuracies=[],[]
    best_acc=0
    best_model=None
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        model.train()
#         for sample in tqdm(train_loader):
        for sample in train_loader:
            X, y = sample['image'], sample['emotion_label_idx']
            X = X.to(device)
#             print(X)
            y = y.to(device)
            output = model(X)

            loss = criterion(output, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc = (output.argmax(dim=1) == y).float().mean()
            
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        epoch_val_accuracy,epoch_val_loss = test(val_loader,model,device,criterion)
#         print(
#             f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
#         )
        train_accuracies.append(epoch_accuracy)
        val_accuracies.append(epoch_val_accuracy)
        if best_acc<epoch_val_accuracy:
            best_acc=epoch_val_accuracy
            best_model=copy.deepcopy(model.state_dict())
        if scheduler is not None:
            scheduler.step()
    
    if best_model is not None:
        model.load_state_dict(best_model)
        print(f"Best acc:{best_acc}")
        epoch_val_accuracy,epoch_val_loss = test(val_loader,model,device,criterion)
        print(
            f"val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
    else:
        print(f"No best model Best acc:{best_acc}")
    return best_model,train_accuracies,val_accuracies         

# Загрузка VGG19 и настройка

## Transfer Learning

In [24]:
model = tv.models.vgg19(weights=tv.models.vgg.VGG19_Weights.IMAGENET1K_V1)

In [25]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [26]:
classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.LeakyReLU(0.2),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(4096, 4096),
    nn.LeakyReLU(0.2),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(4096, 8)
)

count_parameters(classifier) #2.5M параметров получилось

model.classifier = classifier

In [27]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [28]:
for param in model.parameters():
    param.requires_grad = False

In [29]:
for param in model.classifier.parameters():
    param.requires_grad = True

In [30]:
print("Num parameters ALL")
print(count_parameters(model))

Num parameters ALL
119578632


In [31]:
from torch.optim.lr_scheduler import StepLR

In [32]:
n_epochs = 15
model=model.to(device)
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [44]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,model,device,n_epochs,optimizer,criterion,scheduler)

Best acc:0.6238943011297894
val_loss : 1.0849 - val_acc: 0.6239


In [45]:
torch.save(model, "baseline_VGG19_TransferLearning_final.pth")

## Fine Tuning

In [46]:
model = tv.models.vgg19(weights=tv.models.vgg.VGG19_Weights.IMAGENET1K_V1)

In [47]:
classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.LeakyReLU(0.2),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(4096, 4096),
    nn.LeakyReLU(0.2),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(4096, 8)
)

count_parameters(classifier) #2.5M параметров получилось

model.classifier = classifier

In [48]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [49]:
for param in model.parameters():
    param.requires_grad = True

In [50]:
print("Num parameters ALL")
print(count_parameters(model))

Num parameters ALL
139603016


In [51]:
n_epochs = 15
model=model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [52]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,model,device,n_epochs,optimizer,criterion,scheduler)

Best acc:0.6012308710922634
val_loss : 1.0419 - val_acc: 0.6012


In [53]:
torch.save(model, "baseline_VGG19_FineTune_final.pth")

# Загрузка ResNet-18 и настройка

## Transfer Learning

In [56]:
import torchvision

In [57]:
model = torchvision.models.resnet18(weights="IMAGENET1K_V1")

In [58]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [59]:
fc = nn.Sequential(
    nn.Linear(512, 8)
)

In [60]:
count_parameters(fc)

4104

In [61]:
model.fc = fc

In [62]:
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

In [63]:
print("Num parameters ALL")
print(count_parameters(model))

Num parameters ALL
4104


In [64]:
n_epochs = 15
cnn_model=model.to(device)
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [66]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.6327687109942731
val_loss : 1.0794 - val_acc: 0.6328


In [67]:
model_path = "baseline_resnet18_transferlearning_final.pth"
torch.save(cnn_model, model_path)

## Fine Tuning

In [70]:
model = torchvision.models.resnet18(weights="IMAGENET1K_V1")

In [71]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [72]:
fc = nn.Sequential(
    nn.Linear(512, 8)
)

In [73]:
count_parameters(fc)

4104

In [74]:
model.fc = fc

In [75]:
for param in model.parameters():
    param.requires_grad = True

In [76]:
print("Num parameters ALL")
print(count_parameters(model))

Num parameters ALL
11180616


In [77]:
n_epochs = 15
cnn_model=model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [78]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.6720182164982977
val_loss : 1.0160 - val_acc: 0.6720


In [79]:
model_path = "baseline_resnet18_finetuning_final.pth"
torch.save(cnn_model, model_path)

# Загрузка ResNet-50 и настройка

## Transfer Learning

In [80]:
from torchvision import models
cnn_model = models.resnet50(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

C:\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [81]:
num_classes=8
fc = nn.Linear(2048, 8, bias=True)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
119578632
Num parameters ALL
25557032


In [82]:
cnn_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [83]:
for param in cnn_model.parameters():
    param.requires_grad = False

In [84]:
for param in cnn_model.fc.parameters():
    param.requires_grad = True

In [85]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
16392


In [86]:
from torch.optim.lr_scheduler import StepLR

In [87]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [88]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.6673749155639461
val_loss : 1.022 - val_acc: 0.6674


In [89]:
model_path = "baseline_resnet50_transferlearning_final.pth"
torch.save(cnn_model, model_path)

## Fine Tuning

In [90]:
from torchvision import models
cnn_model = models.resnet50(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [91]:
num_classes=8

fc = nn.Linear(2048, 8, bias=True)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc =  fc

Num parameters FC
16392
Num parameters ALL
25557032


In [92]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [93]:
for param in cnn_model.parameters():
    param.requires_grad = True

In [94]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
23524424


In [95]:
from torch.optim.lr_scheduler import StepLR

In [96]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [97]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.7252549872133664
val_loss : 0.9404 - val_acc: 0.7253


In [98]:
model_path = "baseline_resnet50_finetuning_final.pth"

In [99]:
torch.save(cnn_model, model_path)

# Загрузка MobileNet и настройка

## Transfer Learning

In [100]:
from torchvision import models
cnn_model = models.mobilenet_v2(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

C:\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [101]:
num_classes=8
classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8, bias=True)
)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.classifier = classifier

Num parameters FC
10248
Num parameters ALL
3504872


In [102]:
cnn_model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [103]:
for param in cnn_model.parameters():
    param.requires_grad = False

In [104]:
for param in cnn_model.classifier.parameters():
    param.requires_grad = True

In [105]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
10248


In [106]:
from torch.optim.lr_scheduler import StepLR

In [107]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
criterion=nn.CrossEntropyLoss()

In [108]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.6448129745631387
val_loss : 1.0578 - val_acc: 0.6448


In [109]:
model_path = "baseline_mobilenet_transferlearning_final.pth"
torch.save(cnn_model, model_path)

## Fine Tuning

In [110]:
from torchvision import models
cnn_model = models.mobilenet_v2(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [111]:
num_classes=8

classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8, bias=True)
)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.classifier = classifier

Num parameters FC
10248
Num parameters ALL
3504872


In [112]:
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [113]:
for param in cnn_model.parameters():
    param.requires_grad = True

In [114]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
2234120


In [115]:
from torch.optim.lr_scheduler import StepLR

In [116]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
criterion=nn.CrossEntropyLoss()

In [117]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.6923489664213346
val_loss : 0.9854 - val_acc: 0.6923


In [118]:
model_path = "baseline_mobilenet_finetuning_final.pth"
torch.save(cnn_model, model_path)

# Загрузка EfficientNet и настройка

## Transfer Learning

In [119]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

print(efficientnet)

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [120]:
efficientnet

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [121]:
num_classes=8
classifier = nn.Linear(1280, 8)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(efficientnet))

efficientnet.classifier.fc = classifier

Num parameters FC
10248
Num parameters ALL
5288548


In [122]:
efficientnet

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [123]:
for param in efficientnet.parameters():
    param.requires_grad = False

In [124]:
for param in efficientnet.classifier.parameters():
    param.requires_grad = True

In [125]:
print("Num parameters ALL")
print(count_parameters(efficientnet))

Num parameters ALL
10248


In [126]:
from torch.optim.lr_scheduler import StepLR

In [127]:
n_epochs = 15
efficientnet=efficientnet.to(device)
optimizer = torch.optim.Adam(efficientnet.classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
criterion=nn.CrossEntropyLoss()

In [128]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,efficientnet,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.6622573191547346
val_loss : 1.0280 - val_acc: 0.6623


In [129]:
model_path = "baseline_efficientnet_transferlearning_final.pth"
torch.save(cnn_model, model_path)

## Fine Tuning

In [130]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

print(efficientnet)

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [131]:
num_classes=8
#cnn_model.classifier[-1]=nn.Linear(in_features=1280, out_features=num_classes)
# fc = nn.Sequential(
#     nn.Linear(512, 100),
#     nn.LeakyReLU(0.2),
#     nn.Linear(100, 8)
# )

classifier = nn.Linear(1280, 8)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(efficientnet))

efficientnet.classifier.fc = classifier

Num parameters FC
10248
Num parameters ALL
5288548


In [132]:
for param in efficientnet.parameters():
    param.requires_grad = True

In [133]:
print("Num parameters ALL")
print(count_parameters(efficientnet))

Num parameters ALL
4017796


In [134]:
print(efficientnet)

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [135]:
from torch.optim.lr_scheduler import StepLR

In [136]:
n_epochs = 15
efficientnet=efficientnet.to(device)
optimizer = torch.optim.Adam(efficientnet.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [137]:
best_model,train_accuracies,val_accuracies=train(train_loader,val_loader,efficientnet,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.7814933519706512
val_loss : 0.8727 - val_acc: 0.7815


In [138]:
model_path = "baseline_efficientnet_finetuning_final.pth"

In [139]:
torch.save(cnn_model, model_path)